In [1]:
!pip install torch transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import pandas as pd

# ✅ Load dataset from provided CSV file
csv_path = "math_memes_dataset123.csv"
df = pd.read_csv(csv_path)
print(f"✅ Loaded dataset from {csv_path}")

# ✅ Convert dataset to Hugging Face format
dataset = Dataset.from_pandas(df)

# ✅ Format data for training
def format_data(example):
    return {
        "text": f"Wrong Math Meme: {example['input']}\nCorrect Explanation: {example['output']}"
    }

dataset = dataset.map(format_data, remove_columns=["input", "output"])

# ✅ Split dataset (90% train, 10% test)
split_dataset = dataset.train_test_split(test_size=0.1)

# ✅ Print a sample to verify
print(split_dataset["train"][0])

# ✅ Define model & tokenizer (Qwen1.5-4B-Chat)
model_name = "Qwen/Qwen1.5-4B-Chat"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

# ✅ Apply LoRA fine-tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ✅ Ensure tokenizer has a pad token
tokenizer.pad_token = tokenizer.eos_token

# ✅ Tokenization function
def tokenize_data(example):
    tokenized_output = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    labels = tokenized_output["input_ids"].copy()
    labels = [(label if label != tokenizer.pad_token_id else -100) for label in labels]
    tokenized_output["labels"] = labels
    return tokenized_output

# ✅ Apply tokenization
tokenized_datasets = split_dataset.map(tokenize_data, batched=True)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./math_meme_repair_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=9,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
)

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)

# ✅ Start training
trainer.train()

# ✅ Save model and tokenizer
model.save_pretrained("./math_meme_repair_model")
tokenizer.save_pretrained("./math_meme_repair_model")
print("✅ Model and tokenizer saved successfully!")



✅ Loaded dataset from math_memes_dataset123.csv


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

{'text': "Wrong Math Meme: The sum of interior angles of a pentagon is 180° because it's like a triangle\nCorrect Explanation: Wrong! The sum of interior angles of an n-sided polygon is (n–2)×180°. For a pentagon, (5–2)×180° = 540°."}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: absarrashid3 (absarrashid3-nuces) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.335781
2,No log,1.126650
3,2.914400,0.970796
4,2.914400,0.847403
5,0.819300,0.768327
6,0.819300,0.725926
7,0.613100,0.717868
8,0.613100,0.712225
9,0.542600,0.711007


✅ Model and tokenizer saved successfully!


In [4]:
!pip install streamlit transformers bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [6]:
%%writefile your_script.py
import subprocess
import torch
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from pyngrok import ngrok



model_path = "./math_meme_repair_model"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token

# Set quantization to reduce memory usage
quantization_config = BitsAndBytesConfig(load_in_8bit=True)  # Change to load_in_4bit=True if needed

# Load model with device auto-detection and offloading
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",  # Automatically selects best device (GPU if available)
        offload_folder="./offload_dir",  # Offloads large layers to disk if needed
        quantization_config=quantization_config  # Apply quantization
    )
except Exception as e:
    print(f"Failed to load on GPU, switching to CPU: {e}")
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="cpu"  # Fallback to CPU if necessary
    )

# ✅ Streamlit UI
st.title("🤖 SASSY SOLVER")
st.write("Enter a wrong math meme, and the AI will correct it!")

prompt = st.text_input("Enter an incorrect math meme:", "")

if st.button("Fix Math Meme"):
    if prompt:
        input_text = f"Wrong Math Meme: {prompt}\nCorrect Explanation:"
        device = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = tokenizer(input_text, return_tensors="pt").to(device)

        # ✅ Generate response
        output = model.generate(
            **inputs, max_length=60, num_return_sequences=1,
            do_sample=True, temperature=0.7, top_p=0.9
        )
        result = tokenizer.decode(output[0], skip_special_tokens=True)

        st.success(f"📖 Correct Explanation: {result}")
    else:
        st.warning("Please enter a valid math meme.")


Writing your_script.py


In [11]:
import subprocess
import time
from pyngrok import ngrok

# ✅ Start Streamlit in a separate process
process = subprocess.Popen(["streamlit", "run", "your_script.py"])

# ✅ Wait a few seconds to ensure Streamlit is running
time.sleep(5)  # Adjust if needed

# ✅ Expose Streamlit with ngrok
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://06b1-34-168-87-255.ngrok-free.app" -> "http://localhost:8501"


In [10]:
# Ngrok authentication (run only once per machine)
NGROK_AUTH_TOKEN = "2uHP3chnuezzbtUy8KCZpHnZowX_638q32K2y7FMtEDfcc6XH"  # Replace with your token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
import torch

# ✅ More incorrect math memes for testing
test_memes = [
    "0.999... = 1 is false because there's always a tiny gap!",
    "Multiplying by zero makes numbers disappear forever.",
    "If 2+2=4, then 2×2 must equal 8, right?",
    "π is exactly 3.14 because my teacher said so.",
    "Division always makes numbers smaller. 5 ÷ 0 should be 0!",
    "Negative numbers don’t exist because I can’t see them.",
    "2^3 = 6 because 2×3 = 6.",
    "Parallel lines eventually meet if you wait long enough.",
    "1+1 = 3 if you carry the 1 creatively.",
    "Fractions are fake. You can't have half a pizza, it's just two smaller pizzas!"
]

device = "cuda" if torch.cuda.is_available() else "cpu"

for meme in test_memes:
    input_text = f"Wrong Math Meme: {meme}\nCorrect Explanation:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_length=90,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.4,
        top_p=0.9
    )

    fixed_meme = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"❌ {meme} → ✅ {fixed_meme}")

# ✅ Funny error rating
print("🤣 Error Rating: 110% facepalm, 90% chaos!")


❌ 0.999... = 1 is false because there's always a tiny gap! → ✅ Wrong Math Meme: 0.999... = 1 is false because there's always a tiny gap!
Correct Explanation: False! It’s an infinite decimal with no repeating pattern, not equal to 1.
❌ Multiplying by zero makes numbers disappear forever. → ✅ Wrong Math Meme: Multiplying by zero makes numbers disappear forever.
Correct Explanation: Incorrect! Multiplying by zero is the same as 0, and 0 can be added to any number without changing it.
❌ If 2+2=4, then 2×2 must equal 8, right? → ✅ Wrong Math Meme: If 2+2=4, then 2×2 must equal 8, right?
Correct Explanation: Incorrect! Multiplication and addition are separate operations. 2×2 = 4 because 2 × 2 = 4 (not 8).
❌ π is exactly 3.14 because my teacher said so. → ✅ Wrong Math Meme: π is exactly 3.14 because my teacher said so.
Correct Explanation: Pi (π) is an irrational number and cannot be expressed as a simple fraction or decimal. The most common approximation of π is 3.14, but it's not exact.
❌ D

In [ ]:
import torch

# ✅ Test model with new incorrect math memes
test_memes = [
    "52 = 10 because 5×2=10",
    "√25 = ±5 because squaring ±5 gives 25",
    "1/2 + 1/4 = 1/6. Just add denominators!",
]

device = "cuda" if torch.cuda.is_available() else "cpu"

for meme in test_memes:
    input_text = f"Wrong Math Meme: {meme}\nCorrect Explanation:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_length=100,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    fixed_meme = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"❌ {meme} → ✅ {fixed_meme}")

# ✅ Funny error rating
print("🤣 Error Rating: 85% Sass, 15% Patience!")


❌ 52 = 10 because 5×2=10 → ✅ Wrong Math Meme: 52 = 10 because 5×2=10
Correct Explanation: Error! Incorrectly multiplying by 5. To get 52, you need to add two 2-digit numbers with a total of 52 (e.g., 34 + 18).
❌ √25 = ±5 because squaring ±5 gives 25 → ✅ Wrong Math Meme: √25 = ±5 because squaring ±5 gives 25
Correct Explanation: The square root of 25 is 5 because 5 squared equals 25. Square roots are not always positive!
❌ 1/2 + 1/4 = 1/6. Just add denominators! → ✅ Wrong Math Meme: 1/2 + 1/4 = 1/6. Just add denominators!
Correct Explanation: Incorrect! Adding the numerators (1+1=2) would give the correct result of 3/6, or one half.
Corrected Explanation: To add fractions, simply add their numerators and convert to a common denominator if necessary. In this case, 1/2 + 1/4 is equal to 3/4, not
🤣 Error Rating: 85% Sass, 15% Patience!


In [ ]:
import torch

# ✅ New incorrect math memes
test_memes = [
    "0.999... = 1 is fake news. 0.999... is just really close to 1!",
    "Multiplying anything by zero makes it disappear forever.",
    "π = 3.14 exactly, no more digits exist after that.",
    "50% + 50% = 100% always, no matter what!",
    "You can't divide by zero because math teachers said so.",
    "x^2 + y^2 = z^2 is the formula for making pizza slices.",
    "sin(x) is just 'x' written in a fancy way.",
    "If 2^3 = 8, then 3^2 should be 6!",
]

device = "cuda" if torch.cuda.is_available() else "cpu"

for meme in test_memes:
    input_text = f"Wrong Math Meme: {meme}\nCorrect Explanation:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_length=90,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.4,
        top_p=0.9
    )

    fixed_meme = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"❌ {meme} → ✅ {fixed_meme}")

# ✅ Funny error rating
print("🤣 Error Rating: 92% Facepalm, 8% Hope!")


❌ 0.999... = 1 is fake news. 0.999... is just really close to 1! → ✅ Wrong Math Meme: 0.999... = 1 is fake news. 0.999... is just really close to 1!
Correct Explanation: False! While 0.999… is very close to 1, it’s not exactly equal to 1 because of the infinite decimal. In fact, 0.999… approaches 1 as you go on, but it never actually
❌ Multiplying anything by zero makes it disappear forever. → ✅ Wrong Math Meme: Multiplying anything by zero makes it disappear forever.
Correct Explanation: False! Zero is the identity element for multiplication, and any number multiplied by zero remains zero.
❌ π = 3.14 exactly, no more digits exist after that. → ✅ Wrong Math Meme: π = 3.14 exactly, no more digits exist after that.
Correct Explanation: False! π is an irrational number with an infinite decimal expansion (e.g., 3.14159…). It has over 20 digits beyond the familiar 3.14!
❌ 50% + 50% = 100% always, no matter what! → ✅ Wrong Math Meme: 50% + 50% = 100% always, no matter what!
Correct Explanati